In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os 
os.getcwd() 
os.chdir('/content/gdrive/My Drive/app/flutter/flowerdetector')

In [3]:
!ls

trainflowerdetector.ipynb


In [7]:
import tensorflow as tf
from six.moves import urllib

In [8]:
DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
!tar xfz flower_photos.tgz

In [9]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
base_dir = 'flower_photos'
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'training'
)

var_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [10]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('label.txt','w')as f:
  f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [12]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [14]:
base_model.trainable = False

In [15]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32,3,activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5,activation='softmax')

])

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics = ['accuracy'])

In [18]:
epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data = var_generator)

Epoch 1/10
46/46 [==============================] - 15s 334ms/step - loss: 0.6807 - accuracy: 0.7594 - val_loss: 0.4497 - val_accuracy: 0.8440
Epoch 2/10
46/46 [==============================] - 15s 315ms/step - loss: 0.2581 - accuracy: 0.9047 - val_loss: 0.4607 - val_accuracy: 0.8523
Epoch 3/10
46/46 [==============================] - 15s 316ms/step - loss: 0.1642 - accuracy: 0.9411 - val_loss: 0.4055 - val_accuracy: 0.8618
Epoch 4/10
46/46 [==============================] - 15s 317ms/step - loss: 0.1083 - accuracy: 0.9646 - val_loss: 0.4293 - val_accuracy: 0.8577
Epoch 5/10
46/46 [==============================] - 15s 316ms/step - loss: 0.0715 - accuracy: 0.9803 - val_loss: 0.4010 - val_accuracy: 0.8687
Epoch 6/10
46/46 [==============================] - 15s 323ms/step - loss: 0.0435 - accuracy: 0.9912 - val_loss: 0.5422 - val_accuracy: 0.8591
Epoch 7/10
46/46 [==============================] - 14s 313ms/step - loss: 0.0308 - accuracy: 0.9949 - val_loss: 0.5094 - val_accuracy: 0.8673

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 2,626,821
Trainable params: 368,837
Non-trainable params: 2,257,984
_________________________________________________________________


In [25]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite','wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


In [21]:
!ls

assets	       flower_photos.tgz  saved_model.pb	     variables
flower_photos  label.txt	  trainflowerdetector.ipynb
